<a href="https://colab.research.google.com/github/2303A51469/AIML_2303A52431/blob/main/AIML_LAB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1 Implement the AI Game Strategy
Part 1 –(a). Install the Python Libraries required for Game Strategy
1. Install the python libraries - collections, random, math, functools,
cache = functools.lru cache(10**6)
2. Implement a Game Class Constructor using action, is terminal, result, utility functions
3. A game is similar to a problem, but it has a terminal test instead of a goal test, and a
utility for each terminal state.
4. Create a game subclass and implement actions, result, is terminal, and utility.
5. You will also need to set the initial attribute to the initial state; this can be done in the
constructor.

In [ ]:
import collections
import random
import math
from functools import lru_cache

In [ ]:
cache = lru_cache(maxsize=10**6)

class Game:
    def __init__(self, initial_state):
        self.initial_state = initial_state

    def actions(self, state):
        """Return a list of possible actions given the state."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state resulting from taking an action from the current state."""
        raise NotImplementedError

    def is_terminal(self, state):
        """Check if the given state is a terminal state."""
        raise NotImplementedError

    def utility(self, state):
        """Return the utility value of a terminal state."""
        raise NotImplementedError

    def initial_state(self):
        """Return the initial state of the game."""
        return self.initial_state

In [ ]:
class SimpleGame(Game):
    def __init__(self, initial_state):
        super().__init__(initial_state)

    def actions(self, state):
        """Define possible actions based on the state."""
        return ['increment', 'decrement']

    def result(self, state, action):
        """Define the result of applying an action to the state."""
        if action == 'increment':
            return state + 1
        elif action == 'decrement':
            return state - 1
        else:
            raise ValueError("Invalid action")

    def is_terminal(self, state):
        """Define terminal states. For example, a state is terminal if it is 0 or 10."""
        return state == 0 or state == 10

    def utility(self, state):
        """Define utility values for terminal states. Example: 10 if the state is 10, -10 if the state is 0."""
        if state == 10:
            return 10
        elif state == 0:
            return -10
        else:
            raise ValueError("Utility function called on non-terminal state")

In [ ]:
game = SimpleGame(initial_state=5)
state = game.initial_state

print("Initial state:", state)

for action in game.actions(state):
    new_state = game.result(state, action)
    if game.is_terminal(new_state):
        print(f"Action: {action}, New State: {new_state}, Utility: {game.utility(new_state)}")
    else:
        print(f"Action: {action}, New State: {new_state}")

Initial state: 5
Action: increment, New State: 6
Action: decrement, New State: 4


Part-02: Implementation of Game Strategy Algorithm
MiniMax Tree

Alpha-Beta Search Algorithm

In [ ]:
import math

# Define the game tree as a simple dictionary
# Each state is a key, and its possible actions and resulting states are defined
game_tree = {
    5: {'actions': ['increment', 'decrement']},
    6: {'actions': []},   # Terminal states
    4: {'actions': []},   # Terminal states
    10: {'actions': []},  # Terminal states with utility 10
    0: {'actions': []}    # Terminal states with utility -10
}

# Define utilities for terminal states
utilities = {
    10: 10,
    0: -10
}

def alpha_beta_search(initial_state):
    def max_value(state, alpha, beta):
        if state in utilities:
            return utilities[state]

        v = -math.inf
        for action in game_tree[state]['actions']:
            next_state = result(state, action)
            v = max(v, min_value(next_state, alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if state in utilities:
            return utilities[state]

        v = math.inf
        for action in game_tree[state]['actions']:
            next_state = result(state, action)
            v = min(v, max_value(next_state, alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    def result(state, action):
        if action == 'increment':
            return state + 1
        elif action == 'decrement':
            return state - 1
        else:
            raise ValueError("Invalid action")

    best_action = None
    alpha = -math.inf
    beta = math.inf

    for action in game_tree[initial_state]['actions']:
        next_state = result(initial_state, action)
        v = min_value(next_state, alpha, beta)
        if v > alpha:
            alpha = v
            best_action = action

    return best_action

# Example usage
initial_state = 5
best_action = alpha_beta_search(initial_state)
print(f"Best action from state {initial_state} is: {best_action}")

Best action from state 5 is: increment


Part-03: Implement the Game Strategy using TicTacToe

In [ ]:
class TicTacToe(Game):
    """Play TicTacToe on an `height` by `width` board, needing `k` in a row to win.
    'X' plays first against 'O'."""

    def __init__(self, height=3, width=3, k=3):
        self.k = k # k in a row
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0)

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        return self.squares - set(board)

    def result(self, board, square):
        """Place a marker for current player on square."""
        player = board.to_move
        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        win = k_in_row(board, player, square, self.k)
        board.utility = (0 if not win else +1 if player == 'X' else -1)
        return board

    def utility(self, board, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board)

    def display(self, board): print(board)


def k_in_row(board, player, square, k):
    """True if player has k pieces in a line through square."""
    def in_row(x, y, dx, dy): return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)
    return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy)-1>=k
               for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))

In [ ]:
from collections import defaultdict # import the defaultdict class from the collections module

class Board(defaultdict):
    """A board has the player to move, a cached utility value,
    and a dict of {(x, y): player} entries, where player is 'X' or 'O'."""
    empty = '.'
    off = '#'

    def __init__(self, width=8, height=8, to_move=None, **kwds):
        self.__dict__.update(width=width, height=height, to_move=to_move, **kwds)

    def new(self, changes: dict, **kwds) -> 'Board':
        "Given a dict of {(x, y): contents} changes, return a new Board with the changes."
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def __missing__(self, loc):
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off

    def __hash__(self):
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) +  '\n'

In [ ]:
def random_player(game, state): return random.choice(list(game.actions(state)))

def player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)[1]

Evaluate the Game Strategy using TicTokToe

In [ ]:
play_game(TicTacToe(), dict(X=random_player, O=player(alphabeta_search)), verbose=True).utility

Player X move: (1, 2)
. . .
. . .
. X .

Player O move: (1, 1)
. . .
. O .
. X .

Player X move: (2, 1)
. . .
. O X
. X .

Player O move: (2, 0)
. . O
. O X
. X .

Player X move: (0, 1)
. . O
X O X
. X .

Player O move: (1, 0)
. O O
X O X
. X .

Player X move: (0, 0)
X O O
X O X
. X .

Player O move: (0, 2)
X O O
X O X
O X .



-1

In [ ]:
play_game(TicTacToe(), dict(X=player(alphabeta_search), O=player(minimax_search)), verbose=True).utility

Player X move: (0, 1)
. . .
X . .
. . .

Player O move: (2, 1)
. . .
X . O
. . .

Player X move: (1, 2)
. . .
X . O
. X .

Player O move: (0, 0)
O . .
X . O
. X .

Player X move: (1, 1)
O . .
X X O
. X .

Player O move: (1, 0)
O O .
X X O
. X .

Player X move: (2, 0)
O O X
X X O
. X .

Player O move: (0, 2)
O O X
X X O
O X .

Player X move: (2, 2)
O O X
X X O
O X X



0